In [2]:
import pandas as pd
import numpy as np
import re
import datetime as dt

In [132]:
# Checking accident losses
df_acdnt = pd.read_csv('C:/Users/sgulbin/Work/Analysis/ScoringTesting/Accidents_w_user_id_201911.csv')
df_acdnt['user_id'] = df_acdnt['user_id'].fillna(0)
df_acdnt['user_id'] = df_acdnt['user_id'].astype(int)
df_acdnt['Сумма по заказ-наряду'] = df_acdnt['Сумма по заказ-наряду'].fillna(25000)
df_acdnt = df_acdnt[df_acdnt['Виновность'] == 'Виновен']
df_acdnt_grpd = df_acdnt.groupby('user_id').sum()
df_acdnt_grpd = df_acdnt_grpd[['Сумма по заказ-наряду']]
df_check = pd.merge(df, df_acdnt_grpd, left_on = 'user_id', right_on = 'user_id', how = 'left')
df_check['Сумма по заказ-наряду'] = df_check['Сумма по заказ-наряду'].fillna(0)
# print(sum(df_check['Сумма по заказ-наряду']))

In [3]:
# Calculating revenue AS IS
df = pd.read_csv('C:/Users/sgulbin/Desktop/to_serg_final.csv', encoding = 'cp1251')
df['age_rounded'] = df.age.apply(np.floor)
df['exp_rounded'] = df.experience.apply(np.floor)
loss_per_min = -2.21
df['time'] = df.ttime.fillna(0)
df['money'] = df.money.fillna(0)
df['other_losses_AS_IS'] = df.time*loss_per_min
df['accident_losses_AS_IS'] = df.target_amount.fillna(0)*(-1)
df['total_losses_AS_IS'] = df.other_losses_AS_IS+df.accident_losses_AS_IS
df['revenue_AS_IS'] = df.money/1.2
df['avg_min_price'] = df.revenue_AS_IS/df.time
df['profit_AS_IS'] = df.revenue_AS_IS+df.total_losses_AS_IS
# print(df)

In [4]:
# Generating synthetic sex
df['patronymic_name'] = df['patronymic_name'].fillna('Unknown')
df['patronymic_name_end'] = df['patronymic_name'].str.strip().str[-2:]
df.loc[df.patronymic_name_end.str.contains('ич'),'sex_synthetic'] = 'M'
df.loc[df.patronymic_name_end.str.contains('на'),'sex_synthetic'] = 'F'
df.loc[df.patronymic_name_end.str.contains('лы'),'sex_synthetic'] = 'M'
df.loc[df.patronymic_name_end.str.contains('зы'),'sex_synthetic'] = 'F'
df.loc[df.patronymic_name_end.str.contains('wn'),'sex_synthetic'] = 'Unknown'
df['sex_synthetic'] = df['sex_synthetic'].fillna('Unknown')
# print(df)

In [5]:
# Generating age-exp coefficients
K_age_exp_max = 1.2
K_age_exp_min = 0.725
K_intercept = -4.88
K_f_age = 1.26
K_f_exp = 0.08
f_age_exp_min = K_f_age*np.log(19)+K_f_exp*np.log(1)+K_intercept
f_age_exp_max = K_f_age*np.log(50)+K_f_exp*np.log(20)+K_intercept
K = K_age_exp_max-((K_f_age*np.log(19)+K_f_exp*np.log(1)+K_intercept)-f_age_exp_min)*(K_age_exp_max-K_age_exp_min)/(f_age_exp_max-f_age_exp_min)
ages = []
exps = []
koefs = []
for age in range(19,51):
    for exp in range(1,21):
        K = K_age_exp_max-((K_f_age*np.log(age)+K_f_exp*np.log(exp)+K_intercept)-f_age_exp_min)*(K_age_exp_max-K_age_exp_min)/(f_age_exp_max-f_age_exp_min)
        ages.append(age)
        exps.append(exp)
        koefs.append(K)
age_exp_df = pd.DataFrame(data = ({'age':ages, 'exp':exps, 'K_age_exp': koefs}))
# print(age_exp_df)

In [136]:
# Categorizing birth places

bp = pd.read_csv('C:/Users/sgulbin/Desktop/671K_users_birth_place.csv')
bp = bp.fillna('')

bp.loc[bp.birth_place.str.contains('ГОР', flags = re.IGNORECASE,regex = False)|bp.birth_place.str.contains('гор', flags = re.IGNORECASE,regex = False)|
       bp.birth_place.str.contains('ГОР.', flags = re.IGNORECASE,regex = False)|bp.birth_place.str.contains('город', flags = re.IGNORECASE,regex = False)|
       bp.birth_place.str.contains('ГОРОД', flags = re.IGNORECASE,regex = False)|bp.birth_place.str.contains('г.', flags = re.IGNORECASE,regex = False)|
       bp.birth_place.str.contains('Г.', flags = re.IGNORECASE,regex = False),'city'] = 1

bp.loc[bp.birth_place.str.contains('с.', flags = re.IGNORECASE,regex = False)|bp.birth_place.str.contains('С.', flags = re.IGNORECASE,regex = False)|
       bp.birth_place.str.contains('пос.', flags = re.IGNORECASE,regex = False)|bp.birth_place.str.contains('ПОС.', flags = re.IGNORECASE,regex = False)|
       bp.birth_place.str.contains('х.', flags = re.IGNORECASE,regex = False)|bp.birth_place.str.contains('Х.', flags = re.IGNORECASE,regex = False)|
       bp.birth_place.str.contains('пгт.', flags = re.IGNORECASE,regex = False)| bp.birth_place.str.contains('ПГТ.', flags = re.IGNORECASE,regex = False)|
       bp.birth_place.str.contains('пгт', flags = re.IGNORECASE,regex = False)| bp.birth_place.str.contains('ПГТ', flags = re.IGNORECASE,regex = False),'country'] = 1

# bp.loc[bp.birth_place.str.contains('Москва', flags = re.IGNORECASE, regex = False)|
#        bp.birth_place.str.contains('Петербург', flags = re.IGNORECASE,regex = False)|
#        bp.birth_place.str.contains('Ленинград', flags = re.IGNORECASE,regex = False)|
#        bp.birth_place.str.contains('Свердловск', flags = re.IGNORECASE,regex = False)|
#        bp.birth_place.str.contains('Екатеринбург', flags = re.IGNORECASE,regex = False)|
#        bp.birth_place.str.contains('Новосибирск', flags = re.IGNORECASE,regex = False)|
#        bp.birth_place.str.contains('Нижний Новгород', flags = re.IGNORECASE,regex = False)|
#        bp.birth_place.str.contains('Омск', flags = re.IGNORECASE,regex = False)|
#        bp.birth_place.str.contains('Красноярск', flags = re.IGNORECASE,regex = False)|
#        bp.birth_place.str.contains('Уфа', flags = re.IGNORECASE,regex = False)|
#        bp.birth_place.str.contains('Горький', flags = re.IGNORECASE,regex = False)|
#        bp.birth_place.str.contains('Самара', flags = re.IGNORECASE,regex = False)|
#        bp.birth_place.str.contains('Куйбышев', flags = re.IGNORECASE,regex = False)|
#        bp.birth_place.str.contains('Саратов', flags = re.IGNORECASE,regex = False)|
#        bp.birth_place.str.contains('Воронеж', flags = re.IGNORECASE,regex = False),'large_city'] = 1

# bp.loc[bp.birth_place.str.contains('Москва', flags = re.IGNORECASE, regex = False)|
#        bp.birth_place.str.contains('Петербург', flags = re.IGNORECASE, regex = False)|
#        bp.birth_place.str.contains('Ленинград', flags = re.IGNORECASE,regex = False)|
#        bp.birth_place.str.contains('Свердловск', flags = re.IGNORECASE,regex = False)|
#        bp.birth_place.str.contains('Екатеринбург', flags = re.IGNORECASE,regex = False),'big3'] = 1

# bp.loc[bp.birth_place.str.contains('Таджик', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Казах', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Груз', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Армен', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Азерб', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Киргиз', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Кыргыз', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Туркмен', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Узбек', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Дагест', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Ингуш', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Чеч', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Калмык', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Адыг', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Карач', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Кабард', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Осет', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Алания', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Tajik', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Kazakh', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Gruz', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Georg', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Armen', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Azerb', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Kyrgyz', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Turkmen', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Uzbek', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Тбилис', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Tbilis', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Кутаис', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Kutais', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Батум', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Batum', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Рустав', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Rustav', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Цхинвал', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Сухум', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Sukhum', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Ереван', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Yerevan', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Гюмри', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Gyumri', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Ванадзор', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Vanadzor', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Баку', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Baku', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Гяндж', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Gyuandzh', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Душанбе', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Dushanb', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Худжан', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Бохтар', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Ташкент', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Tashkent', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Самарканд', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Samarkand', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Андиж', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Andijon', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Наманган', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Namangan', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Бишкек', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Bishkek', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains(' Ош', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains(' Osh', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Джалал-Абад', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Жалал-Абад', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Jalal-Abad', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Каракол', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Karakol', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Ашхабад', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Ashgabat', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Туркменбаш', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Алма-Ата', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Almaty', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Шымкент', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Shymkent', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Караганда', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Karaganda', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Актобе', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Aktob', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Павлодар', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Pavlodar', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Махачкала', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Владикавказ', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Грозный', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Нальчик', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Пятигорск', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Хасавюрт', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Дербент', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Черкесск', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Назрань', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Каспийск', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Буйнакск', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Буденновск', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Урус-Мартан', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Избербаш', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Шали', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Гудермес', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Кизляр', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Моздок', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Карабулак', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Баксан', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Малгобек', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Аргун', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Кизилюрт', flags = re.IGNORECASE)|
#        bp.birth_place.str.contains('Беслан', flags = re.IGNORECASE),'risky'] = 1

bp.loc[bp.birth_place.str.contains('Дагест', flags = re.IGNORECASE)|
       bp.birth_place.str.contains('Ингуш', flags = re.IGNORECASE)|
       bp.birth_place.str.contains('Чеч', flags = re.IGNORECASE)|
       bp.birth_place.str.contains('Калмык', flags = re.IGNORECASE)|
       bp.birth_place.str.contains('Махачкала', flags = re.IGNORECASE)|
       bp.birth_place.str.contains('Грозный', flags = re.IGNORECASE)|
       bp.birth_place.str.contains('Хасавюрт', flags = re.IGNORECASE)|
       bp.birth_place.str.contains('Дербент', flags = re.IGNORECASE)|
       bp.birth_place.str.contains('Черкесск', flags = re.IGNORECASE)|
       bp.birth_place.str.contains('Назрань', flags = re.IGNORECASE)|
       bp.birth_place.str.contains('Каспийск', flags = re.IGNORECASE)|
       bp.birth_place.str.contains('Буйнакск', flags = re.IGNORECASE)|
       bp.birth_place.str.contains('Буденновск', flags = re.IGNORECASE)|
       bp.birth_place.str.contains('Урус-Мартан', flags = re.IGNORECASE)|
       bp.birth_place.str.contains('Избербаш', flags = re.IGNORECASE)|
       bp.birth_place.str.contains('Шали', flags = re.IGNORECASE)|
       bp.birth_place.str.contains('Гудермес', flags = re.IGNORECASE)|
       bp.birth_place.str.contains('Кизляр', flags = re.IGNORECASE)|
       bp.birth_place.str.contains('Моздок', flags = re.IGNORECASE)|
       bp.birth_place.str.contains('Карабулак', flags = re.IGNORECASE)|
       bp.birth_place.str.contains('Баксан', flags = re.IGNORECASE)|
       bp.birth_place.str.contains('Малгобек', flags = re.IGNORECASE)|
       bp.birth_place.str.contains('Аргун', flags = re.IGNORECASE)|
       bp.birth_place.str.contains('Кизилюрт', flags = re.IGNORECASE)|
       bp.birth_place.str.contains('Элиста', flags = re.IGNORECASE)|
       bp.birth_place.str.contains('Беслан', flags = re.IGNORECASE),'горцы'] = 1

bp['city'] = bp.city.fillna(0)
bp['country'] = bp.country.fillna(0)
bp['large_city'] = bp.large_city.fillna(0)
bp['big3'] = bp.big3.fillna(0)
bp['risky'] = bp.risky.fillna(0)
bp['горцы'] = bp['горцы'].fillna(0)

In [ ]:
# Analyzing birth places

df_bp = pd.merge(df, bp, left_on = 'user_id', right_on = 'user_id', how = 'left')
df_bp = df_bp[['user_id','accident_losses_AS_IS', 'time', 'revenue_AS_IS','city', 'country', 'large_city', 'big3', 'risky', 'горцы']]

city_grpd = df_bp.groupby('city').sum()
country_grpd = df_bp.groupby('country').sum()
large_city_grpd = df_bp.groupby('large_city').sum()
big3_grpd = df_bp.groupby('big3').sum()
risky_grpd = df_bp.groupby('risky').sum()
gorcy_grpd = df_bp.groupby('горцы').sum()

city_grpd = city_grpd[['accident_losses_AS_IS', 'time', 'revenue_AS_IS']]
country_grpd = country_grpd[['accident_losses_AS_IS', 'time', 'revenue_AS_IS']]
large_city_grpd = large_city_grpd[['accident_losses_AS_IS', 'time', 'revenue_AS_IS']]
big3_grpd = big3_grpd[['accident_losses_AS_IS', 'time', 'revenue_AS_IS']]
risky_grpd = risky_grpd[['accident_losses_AS_IS', 'time', 'revenue_AS_IS']]
gorcy_grpd = gorcy_grpd[['accident_losses_AS_IS', 'time', 'revenue_AS_IS']]
                         
grpd1 = pd.merge(city_grpd, country_grpd, left_index=True, right_index=True)
grpd2 = pd.merge(large_city_grpd, big3_grpd, left_index=True, right_index=True)
grpd2 = pd.merge(grpd2, risky_grpd, left_index=True, right_index=True)
grpd = pd.merge(grpd1, grpd2, left_index=True, right_index=True)

# grpd.to_csv('C:/Users/sgulbin/Desktop/birth_place_analysis.csv')

In [7]:
# Adding coefficients to df
df_K_sex = pd.DataFrame(data = ({'sex': ['M', 'F', 'Unknown'], 'K_sex': [1.05,1.10,1.0]}))
df_K_KBM = pd.DataFrame(data = ({'KBM': [0.5,0.55,0.6,0.65,0.7,0.75,0.8,0.85,0.9,0.95,1.0,1.4,1.55,2.3,2.45,np.nan],
                                 'K_KBM': [0.9,0.9,0.9,0.9,0.9,0.9,0.95,0.95,1.0,1.0,1.03,1.1,1.1,1.1,1.1,1.0]}))
df = pd.merge(df, df_K_sex, how = 'left', left_on = 'sex_synthetic', right_on = 'sex')
df = pd.merge(df, df_K_KBM, how = 'left', left_on = 'kbm', right_on = 'KBM')
df = pd.merge(df, age_exp_df, how = 'left', left_on = ['age_rounded','exp_rounded'], right_on = ['age','exp'])
# df = pd.merge(df, bp, how = 'left', left_on = 'user_id', right_on = 'user_id')
df['K_age_exp'] = df['K_age_exp'].fillna(1)

# df.loc[(df['city'] ==1.0)&(df['country'] == 0.0),'K_city'] = 0.9
# df.loc[(df['city'] ==0.0)&(df['country'] == 1.0),'K_city'] = 1.1
# df.loc[(df['city'] ==0.0)&(df['country'] == 0.0),'K_city'] = 1.0
# df['K_горцы'] = np.where(df['горцы'] == 1.0,1.2,0.8)
# df['K_bp'] = df['K_city']*df['K_горцы']

df['K_KBM_model'] = df.K_sex*df.K_KBM*df.K_age_ex#p*df.K_bp
df['K_scoring'] = df.k.fillna(0)
# print(df)

KeyError: 'K_age_exp'

In [139]:
# Calculating final coefficients
df['K_KBM_model_rnd'] = df['K_KBM_model'].clip(0.7,1.3).round(1)
df['K_scoring'] = df.k.fillna(0)
KBM_weight = 0.22
scoring_weight = 0.78
df['KBM_weight'] = np.where(df.k.isnull(),1,KBM_weight)
df['scoring_weight'] = np.where(df.k.isnull(),0,scoring_weight)
df['K_combined'] = df['K_KBM_model_rnd']*df['KBM_weight']+df['K_scoring']*df['scoring_weight']
df['K_combined_rnd'] = df['K_combined'].clip(0.7,1.3).round(1)
# print(df)

In [140]:
# Economics
elast_ty = -1.5
df['KBM_demand_change'] = (df['K_KBM_model_rnd']-1)*elast_ty+1
df['scoring_demand_change'] = (df['K_scoring']-1)*elast_ty+1
df['combined_demand_change'] = (df['K_combined_rnd']-1)*elast_ty+1

df['KBM_time_TO_BE'] = df['time']*df['KBM_demand_change']
df['scoring_time_TO_BE'] = df['time']*df['scoring_demand_change']
df['combined_time_TO_BE'] = df['time']*df['combined_demand_change']

df['KBM_revenue_TO_BE'] = df['KBM_time_TO_BE']*df['avg_min_price']*df['K_KBM_model_rnd']
df['scoring_revenue_TO_BE'] = df['scoring_time_TO_BE']*df['avg_min_price']*df['K_scoring']
df['combined_revenue_TO_BE'] = df['combined_time_TO_BE']*df['avg_min_price']*df['K_combined_rnd']

df['KBM_losses_TO_BE'] = df['KBM_demand_change']*df['total_losses_AS_IS']
df['scoring_losses_TO_BE'] = df['scoring_demand_change']*df['total_losses_AS_IS']
df['combined_losses_TO_BE'] = df['combined_demand_change']*df['total_losses_AS_IS']

df['KBM_profit_TO_BE'] = df['KBM_revenue_TO_BE']+df['KBM_losses_TO_BE']
df['scoring_profit_TO_BE'] = df['scoring_revenue_TO_BE']+df['scoring_losses_TO_BE']
df['combined_profit_TO_BE'] = df['combined_revenue_TO_BE']+df['combined_losses_TO_BE']

df['KBM_profit_diff'] = df['KBM_profit_TO_BE'] - df['profit_AS_IS']
df['scoring_profit_diff'] = df['scoring_profit_TO_BE']-df['profit_AS_IS']
df['combined_profit_diff'] = df['combined_profit_TO_BE']-df['profit_AS_IS']

df['KBM_profit_diff'] = df['KBM_profit_diff'].fillna(0)
df['scoring_profit_diff'] = df['scoring_profit_diff'].fillna(0)
df['combined_profit_diff'] = df['combined_profit_diff'].fillna(0)


In [141]:
# Checking model correctness on users with >= 5 rents

df_less5 = pd.read_csv('C:/Users/sgulbin/Desktop/combined_models_users.csv')
df = pd.merge(df, df_less5, on = 'user_id', how = 'left')
df_to_check = df.dropna(subset = ['K_KBM_model_y'])
# for column in ['K_KBM_model_y', 'K_scoring_y',  'K_combined', 'K_combined_rnd_y', 'KBM_weight_y', 'Scoring_weight']:
#     del df_to_check[column]
# df_to_check.to_csv('C:/Users/sgulbin/Desktop/economic_effect_output.csv')
# print(sum(df.KBM_profit_diff))
# print(sum(df_to_check.scoring_profit_diff))
# print(sum(df.scoring_profit_diff))
print(sum(df.combined_profit_diff))

-1438798.7705833


In [145]:
# KBM Calibration
profit_diffs = []
K_city = []
K_gorcy = []
for i in np.arange(0.85,1.00001,0.025):
    for j in np.arange(1.0,1.150001,0.025):
    
            # Calculating revenue AS IS
            df = pd.read_csv('C:/Users/sgulbin/Desktop/to_serg_final.csv', encoding = 'cp1251')
            df['age_rounded'] = df.age.apply(np.floor)
            df['exp_rounded'] = df.experience.apply(np.floor)
            loss_per_min = -2.21
            df['time'] = df.ttime.fillna(0)
            df['money'] = df.money.fillna(0)
            df['other_losses_AS_IS'] = df.time*loss_per_min
            df['accident_losses_AS_IS'] = df.target_amount.fillna(0)*(-1)
            df['total_losses_AS_IS'] = df.other_losses_AS_IS+df.accident_losses_AS_IS
            df['revenue_AS_IS'] = df.money/1.2
            df['avg_min_price'] = df.revenue_AS_IS/df.time
            df['profit_AS_IS'] = df.revenue_AS_IS+df.total_losses_AS_IS

            # Generating synthetic sex
            df['patronymic_name'] = df['patronymic_name'].fillna('Unknown')
            df['patronymic_name_end'] = df['patronymic_name'].str.strip().str[-2:]
            df.loc[df.patronymic_name_end.str.contains('ич'),'sex_synthetic'] = 'M'
            df.loc[df.patronymic_name_end.str.contains('на'),'sex_synthetic'] = 'F'
            df.loc[df.patronymic_name_end.str.contains('лы'),'sex_synthetic'] = 'M'
            df.loc[df.patronymic_name_end.str.contains('зы'),'sex_synthetic'] = 'F'
            df.loc[df.patronymic_name_end.str.contains('wn'),'sex_synthetic'] = 'Unknown'
            df['sex_synthetic'] = df['sex_synthetic'].fillna('Unknown')

            # Generating age-exp coefficients
            K_age_exp_max = 1.17
            K_age_exp_min = 0.85
            K_intercept = -4.88
            K_f_age = 1.26
            K_f_exp = 0.08
            f_age_exp_min = K_f_age*np.log(19)+K_f_exp*np.log(1)+K_intercept
            f_age_exp_max = K_f_age*np.log(50)+K_f_exp*np.log(20)+K_intercept
            K = K_age_exp_max-((K_f_age*np.log(19)+K_f_exp*np.log(1)+K_intercept)-f_age_exp_min)*(K_age_exp_max-K_age_exp_min)/(f_age_exp_max-f_age_exp_min)
            ages = []
            exps = []
            koefs = []
            for age in range(19,51):
                for exp in range(1,21):
                    K = K_age_exp_max-((K_f_age*np.log(age)+K_f_exp*np.log(exp)+K_intercept)-f_age_exp_min)*(K_age_exp_max-K_age_exp_min)/(f_age_exp_max-f_age_exp_min)
                    ages.append(age)
                    exps.append(exp)
                    koefs.append(K)
            age_exp_df = pd.DataFrame(data = ({'age':ages, 'exp':exps, 'K_age_exp': koefs}))
            
            # Categorizing birth places
            bp = pd.read_csv('C:/Users/sgulbin/Desktop/671K_users_birth_place.csv')
            bp = bp.fillna('')

            bp.loc[bp.birth_place.str.contains('ГОР', flags = re.IGNORECASE,regex = False)|bp.birth_place.str.contains('гор', flags = re.IGNORECASE,regex = False)|
                   bp.birth_place.str.contains('ГОР.', flags = re.IGNORECASE,regex = False)|bp.birth_place.str.contains('город', flags = re.IGNORECASE,regex = False)|
                   bp.birth_place.str.contains('ГОРОД', flags = re.IGNORECASE,regex = False)|bp.birth_place.str.contains('г.', flags = re.IGNORECASE,regex = False)|
                   bp.birth_place.str.contains('Г.', flags = re.IGNORECASE,regex = False),'city'] = 1

            bp.loc[bp.birth_place.str.contains('с.', flags = re.IGNORECASE,regex = False)|bp.birth_place.str.contains('С.', flags = re.IGNORECASE,regex = False)|
                   bp.birth_place.str.contains('пос.', flags = re.IGNORECASE,regex = False)|bp.birth_place.str.contains('ПОС.', flags = re.IGNORECASE,regex = False)|
                   bp.birth_place.str.contains('х.', flags = re.IGNORECASE,regex = False)|bp.birth_place.str.contains('Х.', flags = re.IGNORECASE,regex = False)|
                   bp.birth_place.str.contains('пгт.', flags = re.IGNORECASE,regex = False)| bp.birth_place.str.contains('ПГТ.', flags = re.IGNORECASE,regex = False)|
                   bp.birth_place.str.contains('пгт', flags = re.IGNORECASE,regex = False)| bp.birth_place.str.contains('ПГТ', flags = re.IGNORECASE,regex = False),'country'] = 1

            bp.loc[bp.birth_place.str.contains('Дагест', flags = re.IGNORECASE)|
                   bp.birth_place.str.contains('Ингуш', flags = re.IGNORECASE)|
                   bp.birth_place.str.contains('Чеч', flags = re.IGNORECASE)|
                   bp.birth_place.str.contains('Калмык', flags = re.IGNORECASE)|
                   bp.birth_place.str.contains('Махачкала', flags = re.IGNORECASE)|
                   bp.birth_place.str.contains('Грозный', flags = re.IGNORECASE)|
                   bp.birth_place.str.contains('Хасавюрт', flags = re.IGNORECASE)|
                   bp.birth_place.str.contains('Дербент', flags = re.IGNORECASE)|
                   bp.birth_place.str.contains('Черкесск', flags = re.IGNORECASE)|
                   bp.birth_place.str.contains('Назрань', flags = re.IGNORECASE)|
                   bp.birth_place.str.contains('Каспийск', flags = re.IGNORECASE)|
                   bp.birth_place.str.contains('Буйнакск', flags = re.IGNORECASE)|
                   bp.birth_place.str.contains('Буденновск', flags = re.IGNORECASE)|
                   bp.birth_place.str.contains('Урус-Мартан', flags = re.IGNORECASE)|
                   bp.birth_place.str.contains('Избербаш', flags = re.IGNORECASE)|
                   bp.birth_place.str.contains('Шали', flags = re.IGNORECASE)|
                   bp.birth_place.str.contains('Гудермес', flags = re.IGNORECASE)|
                   bp.birth_place.str.contains('Кизляр', flags = re.IGNORECASE)|
                   bp.birth_place.str.contains('Моздок', flags = re.IGNORECASE)|
                   bp.birth_place.str.contains('Карабулак', flags = re.IGNORECASE)|
                   bp.birth_place.str.contains('Баксан', flags = re.IGNORECASE)|
                   bp.birth_place.str.contains('Малгобек', flags = re.IGNORECASE)|
                   bp.birth_place.str.contains('Аргун', flags = re.IGNORECASE)|
                   bp.birth_place.str.contains('Кизилюрт', flags = re.IGNORECASE)|
                   bp.birth_place.str.contains('Элиста', flags = re.IGNORECASE)|
                   bp.birth_place.str.contains('Беслан', flags = re.IGNORECASE),'горцы'] = 1

            bp['city'] = bp.city.fillna(0)
            bp['country'] = bp.country.fillna(0)
            bp['горцы'] = bp['горцы'].fillna(0)

            # Adding coefficients to df
            df_K_sex = pd.DataFrame(data = ({'sex': ['M', 'F', 'Unknown'], 'K_sex': [1.05,1.11,1.0]}))
            df_K_KBM = pd.DataFrame(data = ({'KBM': [0.5,0.55,0.6,0.65,0.7,0.75,0.8,0.85,0.9,0.95,1.0,1.4,1.55,2.3,2.45,np.nan],
                                             'K_KBM': [0.9,0.9,0.9,0.9,0.9,0.9,0.95,0.95,1.0,1.0,1.03,1.1,1.1,1.1,1.1,1.0]}))
            df = pd.merge(df, df_K_sex, how = 'left', left_on = 'sex_synthetic', right_on = 'sex')
            df = pd.merge(df, df_K_KBM, how = 'left', left_on = 'kbm', right_on = 'KBM')
            df = pd.merge(df, age_exp_df, how = 'left', left_on = ['age_rounded','exp_rounded'], right_on = ['age','exp'])
            df = pd.merge(df, bp, how = 'left', left_on = 'user_id', right_on = 'user_id')
            df['K_age_exp'] = df['K_age_exp'].fillna(1)

            df.loc[(df['city'] ==1.0)&(df['country'] == 0.0),'K_city'] = i
            df.loc[(df['city'] ==0.0)&(df['country'] == 1.0),'K_city'] = 2.0-i
            df.loc[(df['city'] ==0.0)&(df['country'] == 0.0),'K_city'] = 1.0
            df['K_горцы'] = np.where(df['горцы'] == 1.0,j,2.0-j)
            df['K_bp'] = df['K_city']*df['K_горцы']

            df['K_KBM_model'] = df.K_sex*df.K_KBM*df.K_age_exp*df.K_bp
            df['K_scoring'] = df.k.fillna(0)

            # Calculating final coefficients
            df['K_KBM_model_rnd'] = df['K_KBM_model'].clip(0.7,1.3).round(1)
            df['K_scoring'] = df.k.fillna(0)
            KBM_weight = 0.22
            scoring_weight = 1-KBM_weight
            df['KBM_weight'] = np.where(df.k.isnull(),1,KBM_weight)
            df['scoring_weight'] = np.where(df.k.isnull(),0,scoring_weight)
            df['K_combined'] = df['K_KBM_model_rnd']*df['KBM_weight']+df['K_scoring']*df['scoring_weight']
            df['K_combined_rnd'] = df['K_combined'].clip(0.7,1.3).round(1)

            # Economics TO BE
            elast_ty = -1.5
            df['KBM_demand_change'] = (df['K_KBM_model_rnd']-1)*elast_ty+1
            df['scoring_demand_change'] = (df['K_scoring']-1)*elast_ty+1
            df['combined_demand_change'] = (df['K_combined_rnd']-1)*elast_ty+1

            df['KBM_time_TO_BE'] = df['time']*df['KBM_demand_change']
            df['scoring_time_TO_BE'] = df['time']*df['scoring_demand_change']
            df['combined_time_TO_BE'] = df['time']*df['combined_demand_change']

            df['KBM_revenue_TO_BE'] = df['KBM_time_TO_BE']*df['avg_min_price']*df['K_KBM_model_rnd']
            df['scoring_revenue_TO_BE'] = df['scoring_time_TO_BE']*df['avg_min_price']*df['K_scoring']
            df['combined_revenue_TO_BE'] = df['combined_time_TO_BE']*df['avg_min_price']*df['K_combined_rnd']

            df['KBM_losses_TO_BE'] = df['KBM_demand_change']*df['total_losses_AS_IS']
            df['scoring_losses_TO_BE'] = df['scoring_demand_change']*df['total_losses_AS_IS']
            df['combined_losses_TO_BE'] = df['combined_demand_change']*df['total_losses_AS_IS']

            df['KBM_profit_TO_BE'] = df['KBM_revenue_TO_BE']+df['KBM_losses_TO_BE']
            df['scoring_profit_TO_BE'] = df['scoring_revenue_TO_BE']+df['scoring_losses_TO_BE']
            df['combined_profit_TO_BE'] = df['combined_revenue_TO_BE']+df['combined_losses_TO_BE']

            df['KBM_profit_diff'] = df['KBM_profit_TO_BE'] - df['profit_AS_IS']
            df['scoring_profit_diff'] = df['scoring_profit_TO_BE']-df['profit_AS_IS']
            df['combined_profit_diff'] = df['combined_profit_TO_BE']-df['profit_AS_IS']

            df['KBM_profit_diff'] = df['KBM_profit_diff'].fillna(0)
            df['scoring_profit_diff'] = df['scoring_profit_diff'].fillna(0)
            df['combined_profit_diff'] = df['combined_profit_diff'].fillna(0)
            print(sum(df.KBM_profit_diff))
            print(i,j)
            profit_diffs.append(sum(df.KBM_profit_diff))
            K_city.append(i)
            K_gorcy.append(j)

summary = pd.DataFrame(data = ({'profit_diff':profit_diffs, 'K_city':K_city,'K_gorcy':K_gorcy}))
print(summary)
summary.to_csv('C:/Users/sgulbin/Desktop/weights_summary.csv')


-7112916.665890045
0.85 1.0
-9863360.262069196
0.85 1.025
-13480024.216538632
0.85 1.0499999999999998
-17540933.808756307
0.85 1.0749999999999997
-22772402.60694039
0.85 1.0999999999999996
-27592027.73493358
0.85 1.1249999999999996
-32975449.718785286
0.85 1.1499999999999995
-4531315.645678018
0.875 1.0
-7143502.3890584735
0.875 1.025
-10286003.265874896
0.875 1.0499999999999998
-14249209.282810977
0.875 1.0749999999999997
-19422134.007760834
0.875 1.0999999999999996
-24972477.75887206
0.875 1.1249999999999996
-30256556.97147681
0.875 1.1499999999999995
-2214633.9423629907
0.9 1.0
-4835185.462787646
0.9 1.025
-7668716.674051759
0.9 1.0499999999999998
-11230955.03728573
0.9 1.0749999999999997
-16048374.939999288
0.9 1.0999999999999996
-21768434.298198756
0.9 1.1249999999999996
-27684694.87022481
0.9 1.1499999999999995
-346375.01909599366
0.925 1.0
-2615985.280522913
0.925 1.025
-5403223.91652274
0.925 1.0499999999999998
-8822758.081456201
0.925 1.0749999999999997
-13575368.625300178
0.9